In [1]:
import pandas as pd
import numpy as np
import pickle
import re
import itertools
import os

import requests
import urllib2  
from bs4 import BeautifulSoup 
import html5lib
import json
import shutil
from StringIO import StringIO
import gzip 
import time

from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

In [2]:
driver = webdriver.Chrome()
driver.get('https://www.pinterest.com/')

In [3]:
links = ['https://www.pinterest.com/search/pins/?q=spaghetti%20bolognese&rs=typed&0=spaghetti%7Ctyped&1=bolognese%7Ctyped',
         'https://www.pinterest.com/search/pins/?q=zoodles&rs=typed&0=zoodles%7Ctyped']

for link in links:

    driver.get(link)
    srcs = []
    no_of_scrolls = 200

    while no_of_scrolls:

        time.sleep(5)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        no_of_scrolls-=1

        if no_of_scrolls % 20 == 0:

            time.sleep(10)
            html_source = driver.page_source
            data = html_source.encode('utf-8')
            time.sleep(5)

            soup = BeautifulSoup(data)
            srcs.append([x['src'] for x in soup.findAll('img', {'class': 'pinImg fullBleed loaded fade'})])

        print str(no_of_scrolls) + '-' + str(len(srcs))

    flatList = [val for sublist in srcs for val in sublist] 
    finalsrcs = set(flatList)
    finalsrcs = list(finalsrcs)

    for i in range(0,len(finalsrcs)):
        response = requests.get(finalsrcs[i], stream=True)
        if response.status_code == 200:
            with open(str(i) + '.jpg', 'wb') as f:
                response.raw.decode_content = True
                shutil.copyfileobj(response.raw, f) 

In [4]:
flatList = [val for sublist in srcs for val in sublist] 
finalsrcs = set(flatList)
finalsrcs = list(finalsrcs)

for i in range(0,len(finalsrcs)):
    response = requests.get(finalsrcs[i], stream=True)
    if response.status_code == 200:
        with open(str(i) + '.jpg', 'wb') as f:
            response.raw.decode_content = True
            shutil.copyfileobj(response.raw, f) 